In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install seaborn statsmodels brian2 spykes

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
PATH = '../../'
sys.path.append(f'{PATH}')
#sys.path.append(f'{PATH}/npx_utils')
#sys.path.append(f'{PATH}/neb_utils')
import npx_utils.readSGLX as readSGLX
import seaborn as sns
from pathlib import Path
from scipy.signal import savgol_filter
import npx_utils.post_sort_proc as postproc
from npx_utils.data import get_ni_analog,get_tvec,create_spike_dict
%pylab
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [ ]:
?os.path.relpath

In [ ]:
PATH = '/active/ramirez_j/ramirezlab/nbush/projects'
run_path = f'{PATH}/dynaresp/data/processed/m2020-12/m202012_g11'
imec_path = f'{run_path}/m202012_g11_imec0'
ni_fn = f'{run_path}/m202012_g11_t0.nidq.bin'
sync_ts = np.loadtxt(f'{imec_path}/m202012_g11_tcat.imec0.SY_384_6_500.txt')
resp_chan = 1


In [ ]:
sample_rate = 3.000032e+04


In [ ]:
p_save = '/active/ramirez_j/ramirezlab/nbush/projects/dynaresp/results/2020-08-21-npx-sc-analysis'

In [ ]:
st = np.load(os.path.join(imec_path,'spike_times_sec.npy'))
sc = np.load(os.path.join(imec_path,'spike_clusters.npy'))

In [ ]:
meta=readSGLX.readMeta(Path(ni_fn))
ni_dat = readSGLX.makeMemMapRaw(ni_fn,meta)
resp = get_ni_analog(ni_fn,1)
sync_chan = get_ni_analog(ni_fn,0)
ni_sr = readSGLX.SampRate(meta)
tvec = get_tvec(sync_chan,sync_ts,ni_sr)

### Look at nasal trace

In [ ]:
resp_smooth = savgol_filter(resp,501,1)


In [ ]:
plt.plot(tvec,resp,'k',alpha=.2)
plt.plot(tvec,resp_smooth)
plt.xlim(10,12)


In [ ]:
phi = postproc.nasal_to_phase(resp_smooth- np.mean(resp_smooth))
insp = postproc.get_insp_onset(resp_smooth,direction=-1,thresh=2)
insp_sec = tvec[insp]

In [ ]:
phi_s = postproc.shift_phi(phi,insp)

In [ ]:
plt.plot(tvec,phi,lw=1,ls='--')
plt.plot(tvec,phi_s,lw=3)
plt.plot(tvec,(resp_smooth-np.mean(resp_smooth))*100)
plt.axhline(0,color='k',ls=':')
plt.vlines(insp_sec,-np.pi,np.pi,ls='--')
plt.xlim(10,12)
plt.ylim(-np.pi,np.pi)
sns.despine();

In [ ]:
# plt.plot(tvec[2:],-np.diff(savgol_filter(np.diff(resp_smooth),501,1)))
f = plt.figure(figsize=(5,2))
plt.plot(tvec,resp_smooth,'k')
plt.plot(insp_sec,resp_smooth[insp],'ro',alpha=0.4)
plt.xlim(50,70)
plt.savefig(os.path.join(p_save,'resp_trace_detect.png'),dpi=300)


### Load spikes and inspect

In [ ]:
spike_dict = create_spike_dict(imec_path)

In [ ]:
df = pd.DataFrame.from_dict(spike_dict,orient='index')
df.index.name='cluster'
df_good = df[df.label=='good'].sort_values('depth')
df_good.reset_index()

## Smoothed Spike Rates as Heatmap

In [ ]:
import neb_utils.brian_utils.postproc as bup

In [ ]:
raster,cell_id,bins = bup.bin_trains(st*bup.second,sc,len(np.unique(sc)),max_time=tvec[-1]*bup.second,binsize=50*bup.ms);

In [ ]:
depth_order = np.argsort(df.depth.values)
depth = np.sort(df.depth.values)

In [ ]:
f = plt.figure(figsize=(3,10))
plt.pcolormesh(bins,cell_id,raster[depth_order,:],cmap='gray_r')
plt.xlim(100,105)
plt.xlabel('Time (s)')
plt.ylabel('Neuron ID')

### Smoothed Spike Rates as Traces

In [ ]:

f = plt.figure(figsize=(4,10))

for clu in df_good.index.unique():
    depth = df.iloc[clu]['depth']
    fr = raster[clu,:]
    plt.plot(bins,fr*10+depth,'k',lw=0.5,alpha=0.8)

plt.plot(tvec,resp_smooth*10000+3500)
plt.xlim(100,104)
plt.ylim(0,3500)
plt.ylabel('Depth (uM)')
plt.xlabel('Time (s)')
sns.despine()
plt.savefig(os.path.join(p_save,'population_rates_by_depth.png'),dpi=300)


### Work with Pavan's spykes package to visualize

In [ ]:
from spykes.plot.neurovis import NeuroVis
from spykes.plot.popvis import PopVis

In [ ]:
neuron_list=  []
for ii in df_good.index:
    ts = df_good.loc[ii]['ts']
    neuron = NeuroVis(ts,name=f'{ii}')
    neuron_list.append(neuron)

In [ ]:
pop = PopVis(neuron_list)

#### Align to inspiration time (red line in figure)

In [ ]:
data_df = pd.DataFrame()
data_df['insp'] = tvec[insp]

In [ ]:
all_psth = pop.get_all_psth(event='insp',df=data_df,binsize=1,window=[-100,250],plot=False)

In [ ]:
depth_order = np.argsort(df_good.depth.values)
plt.figure(figsize=(3,5))
pop.plot_heat_map(all_psth,sortby=depth_order,normalize='each')
plt.gca().set_yticklabels([])
plt.title('Ordered by depth\nNormalized FR')
plt.savefig(os.path.join(p_save,'pop_psth.png'),dpi=300)
plt.close('all')

In [ ]:
f = pop.plot_heat_map(all_psth,sortby='latency',normalize='each')
plt.gca().set_yticks([])

In [ ]:
mod = []
for neuron in neuron_list:
    psth = neuron.get_psth(event='insp',df=data_df,plot=False,binsize=3);
    max_fr = np.max(psth['data'][0]['mean'])
    min_fr = np.min(psth['data'][0]['mean'])
    mean_sem = np.mean(psth['data'][0]['sem'])
    mod.append((max_fr-min_fr)/mean_sem)
mod = np.array(mod)

In [ ]:
best_list = np.argsort(mod)[::-1][:15]
worst_list = np.argsort(mod)[::-1][-15:]

### Plot all best

In [ ]:
f = plt.figure(figsize=(14,8))
count = 1
for ii in best_list:
    ax = f.add_subplot(4,4,count)
    neuron_list[ii].get_psth(event='insp',df=data_df,binsize=5,window=[-400,400]);
    count+=1
    ax.set_ylim(0,220)
plt.tight_layout()
plt.savefig(os.path.join(p_save,f'best_tuning_mods.png'),dpi=300)
plt.close('all')

### Plot all worst

In [ ]:
f = plt.figure(figsize=(14,8))
count = 1
for ii in worst_list:
    ax = f.add_subplot(4,4,count)
    neuron_list[ii].get_psth(event='insp',df=data_df,binsize=5,window=[-400,400]);
    count+=1
    ax.set_ylim(0,220)
plt.tight_layout()
plt.savefig(os.path.join(p_save,f'worst_tuning_mods.png'),dpi=300)
plt.close('all')

### Plot best individual

In [ ]:
count = 1
for ii in best_list:
    f = plt.figure()
    neuron_list[ii].get_psth(event='insp',df=data_df,binsize=5,window=[-400,400]);
    plt.savefig(os.path.join(p_save,f'tuning_cuve-mod{mod[ii]:2.0f}-neuron{ii}.png'),dpi=300)
    plt.close('all')

### Plot modulation by depth

In [ ]:
depth = df_good['depth'].values

In [ ]:

hist,edges = np.histogram(depth,bins=np.arange(0,3500,200),weights=mod)

In [ ]:
import seaborn as sns

In [ ]:
df2 = pd.DataFrame()
df2['edges'] = edges[1:]
df2['hist'] = hist

In [ ]:
plt.figure(figsize=(2,4.5))
plt.barh(edges[1:],hist/len(hist),height=200,alpha=.4,color='k')
plt.plot(mod,depth,'o',markeredgecolor='k',markerfacecolor='r',alpha=0.2)
sns.despine()
plt.xlabel('Modulation (a.u.)')
plt.ylabel('Distance from Caudal insertion (microns)')
plt.tight_layout()
plt.savefig(os.path.join(p_save,'depth_vs_modulation.png'),dpi=300)



In [ ]:
plt.close('all')

### Optotag?

In [ ]:
from neb_utils.ephys.signal import binary_onsets
opto = get_ni_analog(ni_fn,3)
light_on,light_off = binary_onsets(opto,1)
light_on = tvec[light_on]
light_of = tvec[light_off]

#### Align to optic signal

In [ ]:
opto_df = pd.DataFrame()
opto_df['light_on'] = light_on

In [ ]:
psth = pop.get_all_psth(df=opto_df,event='light_on',window=[-15,50],plot=False,binsize=1);
pop.plot_heat_map(psth)

In [ ]:
tagged = np.argmax(np.max(psth['data'][0],1))
print(f'Opto tagged neuron is #{tagged}')

In [ ]:
raster = neuron_list[82].get_raster(df= opto_df,event='light_on',window=[-20,20],binsize=1);

In [ ]:
psth = neuron_list[82].get_psth(event='light_on',df=opto_df,binsize=3,window=[-20,20]);

In [ ]:
psth = neuron_list[82].get_psth(event='insp',df=data_df,binsize=3);

### Plot by phase

In [ ]:
from tqdm import tqdm

In [ ]:
all_rate = []
for neuron in tqdm(neuron_list):
    st = neuron.spiketimes

    sp_samp= np.searchsorted(tvec,st)
    sp = np.zeros_like(phi_s)
    sp[sp_samp] = 1
    rate,theta_k,theta,L_dir = postproc.angular_response_hist(phi_s,sp,100)
    all_rate.append(rate)
all_rate = np.array(all_rate)

In [ ]:
normed_rate = all_rate.T/np.max(all_rate,1)
normed_rate = normed_rate.T
order = np.argsort(np.argmax(all_rate,1))
order_r = np.argsort(np.argmin(all_rate,1))
# order = np.argsort(df_good.depth)

In [ ]:

f = plt.figure(figsize=(4,8))
plt.pcolormesh(theta_k[:-1],np.arange(len(neuron_list)),normed_rate[order,:],
              cmap='gray_r')
plt.axvline(0,c='r',ls=':')
plt.xticks([-np.pi, -np.pi/2,0,np.pi/2,np.pi])
plt.gca().set_xticklabels(['-pi','-pi/2','0','pi/2','pi'])
plt.colorbar()

In [ ]:
for ii in np.arange(30,90,5):
    plt.plot(theta_k[:-1],all_rate[ii],alpha=.6)

In [ ]:
all_theta = []
all_L = []
for rate in all_rate:
    theta,L = postproc.get_PD_from_hist(theta_k[:-1],rate)
    all_theta.append(theta)
    all_L.append(L)
all_theta = np.array(all_theta)
all_L = np.array(all_L)

In [ ]:
f,ax = plt.subplots(nrows=1,ncols=2)
ax[0].hist(all_theta)
ax[1].hist(all_L)
sns.despine();

In [ ]:
df = pd.DataFrame()
df['sig'] = all_L>0.05
df['all_L'] = all_L
df['phi_pref'] = all_theta
df['depth'] = df_good.depth.values
df_sig = df[df.sig]
df_notsig = df[df.sig==False]
order = df_sig.sort_values('phi_pref').index
order_r = df_notsig.sort_values('phi_pref').index

In [ ]:
df

In [ ]:
f = plt.figure(figsize=(4,8))
plt.pcolormesh(theta_k[:-1],np.arange(len(order)),all_rate[order,:],
              cmap='viridis',vmax=0.02)
plt.axvline(0,c='r',ls=':')
plt.xticks([-np.pi, -np.pi/2,0,np.pi/2,np.pi])
plt.gca().set_xticklabels(['-pi','-pi/2','0','pi/2','pi'])
plt.colorbar()

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
xx = df.sort_values('depth')['depth'].values
yy= lowess(df['all_L'].values,xx,frac=0.15,return_sorted=False)

In [ ]:
plt.figure(figsize=(2,5))
plt.plot(df['all_L'],df["depth"],'ko',alpha=0.4)
plt.plot(yy,xx,'r--',lw=3)

In [ ]:

dff = df.dropna()
plt.hist2d(dff['phi_pref'],dff.depth);